In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel, LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
parsed_data = [LabeledPoint(0.0, [4.6,3.6,1.0,0.2]),
               LabeledPoint(0.0, [5.7,4.4,1.5,0.4]),
               LabeledPoint(1.0, [6.7,3.1,4.4,1.4]),
               LabeledPoint(0.0, [4.8,3.4,1.6,0.2]),
               LabeledPoint(2.0, [4.4,3.2,1.3,0.2])]     
#model = LogisticRegressionWithSGD.train(sc.parallelize(parsed_data)) # gives error:
# org.apache.spark.SparkException: Input validation failed.
model = LogisticRegressionWithLBFGS.train(sc.parallelize(parsed_data), numClasses=3)  # works OK

In [1]:
import pyspark
sc = pyspark.SparkContext()

In [80]:
import sqlite3
import datetime
import time
from time import strptime
from datetime import datetime

from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import RandomForest


bd = sqlite3.connect('velib3.db')
c = bd.cursor()

def feedData(jour, mois, annee, num_station):

    #select * from station_dyn where num = 10114  and db_write_time >= ('16/03/2017') and db_write_time < ('17/03/2017') order by last_update

    enregistrements=[]

    week_day = datetime(annee, mois, jour).timetuple()[6]
    if week_day not in (5,6):
        hist=[-1,-2,-3,-7,-14]
    else:
        hist=[-7,-8,-14,-15]
        if week_day == 6:
            hist.append(-1)

    for i in hist:
        if datetime(annee, mois, jour+i).timetuple()[6] in (5,6):
            hist.append(i-2)
            continue
        clause ="last_update_date >= (\'"+str(datetime(annee, mois, jour+i))[:10]+"\') and last_update_date < (\'"+ str(datetime(annee, mois, jour+i+1))[:10]+"\')"
        c.execute("select nb_velo_dispo, last_update, last_update_date from station_dyn where "+clause+" and num = "+str(num_station)+" order by last_update")
        enregistrements.extend(c.fetchall())
        enregistrements=sorted(enregistrements, key=lambda e: e[1])
    print enregistrements
    #print len(enregistrements)
    
    return enregistrements

#test = feedData(28, 3, 2017, 15003)
#print test

c.execute("select DISTINCT num from station_dyn" )

list_station=[]

for row in c:
    list_station.append(row[0]) 

"""
for h in range(0,24):
    for m in range(0,51,10):
        #print ("0" if len(str(h))==1 else "")+str(h)+":"+str(m)+("0" if m==0 else "") #144 intervalles par jour
        pass"""

data=[]
interval_pred=600*3

fichier = open("data.txt", "a")
i=0
for e in range(0,10):
    i+=1
    retour_chariot = '\n' if i!=1 else ""
    fichier.write(retour_chariot+"Bonjour monde")
fichier.close()

for station in range(15003,15004):#0,1):#len(list_station)):
    list_row=feedData(28, 3, 2017, 15003)
    
    #c.execute("select nb_velo_dispo, last_update, last_update_date from station_dyn where num = "+str(15003)+" order by last_update")
    #c.execute("select nb_velo_dispo, last_update, last_update_date from station_dyn where num = "+str(list_station[station])+" order by last_update")
    
    #list_row=c.fetchall()
    
    """i=0
    while list_row!=[]:
        print list_row.pop(0)
        i+=1
        print str(i)+(" vide" if list_row==[] else " non vide")
    print len(list_row)
    break"""
    
    #print list_row[-1]
    #print time.strftime("%d/%m/%Y-%H:%M:%S", time.gmtime((int(list_row[-1][1])/1000/600)*600))
    #print time.strftime("%d/%m/%Y-%H:%M:%S", time.gmtime((int(list_row[0][1])/1000/600)*600+600))
    
    row=1
    boucle=0
    in_flow=0
    out_flow=0
    date_interval = (int(list_row[0][1])/1000/interval_pred)*interval_pred+interval_pred
    #for date in range((int(list_row[0][1])/1000/600)*600+600,(int(list_row[-1][1])/1000/600)*600+1,600):
        #print datetime.datetime.fromtimestamp(date).strftime('%H:%M'), datetime.datetime.fromtimestamp(date).strftime('%d-%m')
        
        #implémenter les filtres
    prec_row=list_row[0]
    current_row=list_row.pop(0) #Erreur si la liste est vide
    while list_row!=[]:
        current_row=list_row.pop(0)
        row+=1
        
        flow=current_row[0]-prec_row[0]
        
        if int(current_row[1])/1000<date_interval:
            boucle+=1
            in_flow += flow if flow>0 else 0
            out_flow += flow if flow<0 else 0
        else: #in_flow : nb_velo_dispo, week-day, heure_normalisée
            #data.append(LabeledPoint(float(in_flow),[float(current_row[0]),time.gmtime(int(current_row[1])/1000)[6],int(time.strftime("%H", time.gmtime(date_interval+3600-interval_pred)))*2+int(time.strftime("%M", time.gmtime(date_interval+3600-interval_pred)))/30]))
            #=>data.append(LabeledPoint(float(in_flow),[time.gmtime(int(current_row[1])/1000)[6],int(time.strftime("%H", time.gmtime(date_interval+3600-interval_pred)))*2+int(time.strftime("%M", time.gmtime(date_interval+3600-interval_pred)))/30]))

            data.append(LabeledPoint(float(in_flow),[int(time.strftime("%H", time.gmtime(date_interval+3600-interval_pred)))*2+int(time.strftime("%M", time.gmtime(date_interval+3600-interval_pred)))/30]))

            #data.append(LabeledPoint(float(in_flow),[float((time.gmtime(int(current_row[1])/1000)[6]))/6,float((int(time.strftime("%H", time.gmtime(date_interval+3600-interval_pred)))*2+int(time.strftime("%M", time.gmtime(date_interval+3600-interval_pred)))/30))/47]))

            print [LabeledPoint(float(in_flow),\
                   [time.gmtime(int(current_row[1])/1000)[6],\
                   int(time.strftime("%H", time.gmtime(date_interval+3600-interval_pred)))*6+int(time.strftime("%M", time.gmtime(date_interval+3600-interval_pred)))/10])]
            while int(current_row[1])/1000>=date_interval:
                date_interval+=interval_pred
            boucle=1
            in_flow = flow if flow>0 else 0
            out_flow = flow if flow<0 else 0
        
        print boucle, row, current_row, time.strftime("%H:%M:%S", time.gmtime((int(current_row[1])/1000+3600)-(date_interval+3600-interval_pred)))
        print "intervalle : ["+str(time.strftime("%H:%M", time.gmtime(date_interval+3600-interval_pred)))+","+str(time.strftime("%H:%M", time.gmtime(date_interval+3600)))+"["+" -  n° : "+str(int(time.strftime("%H", time.gmtime(date_interval+3600-interval_pred)))*2+int(time.strftime("%M", time.gmtime(date_interval+3600-interval_pred)))/30) #doit être modifié à la main si changemenr d'intervalle
        
        print "in flow : "+str(in_flow)+" - out flow : "+str(out_flow)
        
        prec_row=current_row

c.close()

[(0, u'1489452485000', u'2017-03-14 01:48:05'), (0, u'1489453088000', u'2017-03-14 01:58:08'), (0, u'1489453692000', u'2017-03-14 02:08:12'), (0, u'1489454295000', u'2017-03-14 02:18:15'), (0, u'1489454899000', u'2017-03-14 02:28:19'), (0, u'1489455502000', u'2017-03-14 02:38:22'), (0, u'1489456106000', u'2017-03-14 02:48:26'), (0, u'1489456710000', u'2017-03-14 02:58:30'), (0, u'1489457297000', u'2017-03-14 03:08:17'), (0, u'1489457482000', u'2017-03-14 03:11:22'), (0, u'1489457491000', u'2017-03-14 03:11:31'), (0, u'1489458094000', u'2017-03-14 03:21:34'), (0, u'1489458698000', u'2017-03-14 03:31:38'), (0, u'1489459302000', u'2017-03-14 03:41:42'), (0, u'1489459906000', u'2017-03-14 03:51:46'), (0, u'1489460509000', u'2017-03-14 04:01:49'), (0, u'1489461113000', u'2017-03-14 04:11:53'), (0, u'1489461716000', u'2017-03-14 04:21:56'), (0, u'1489462320000', u'2017-03-14 04:32:00'), (0, u'1489462923000', u'2017-03-14 04:42:03'), (0, u'1489463527000', u'2017-03-14 04:52:07'), (0, u'148946

In [81]:
print len(data)
print "********"
print data

235
********
[LabeledPoint(0.0, [3.0]), LabeledPoint(0.0, [4.0]), LabeledPoint(0.0, [5.0]), LabeledPoint(0.0, [6.0]), LabeledPoint(0.0, [7.0]), LabeledPoint(0.0, [8.0]), LabeledPoint(0.0, [9.0]), LabeledPoint(0.0, [10.0]), LabeledPoint(2.0, [11.0]), LabeledPoint(1.0, [12.0]), LabeledPoint(1.0, [13.0]), LabeledPoint(5.0, [14.0]), LabeledPoint(9.0, [15.0]), LabeledPoint(8.0, [16.0]), LabeledPoint(8.0, [17.0]), LabeledPoint(9.0, [18.0]), LabeledPoint(13.0, [19.0]), LabeledPoint(1.0, [20.0]), LabeledPoint(2.0, [21.0]), LabeledPoint(2.0, [22.0]), LabeledPoint(1.0, [23.0]), LabeledPoint(7.0, [24.0]), LabeledPoint(2.0, [25.0]), LabeledPoint(1.0, [26.0]), LabeledPoint(4.0, [27.0]), LabeledPoint(7.0, [28.0]), LabeledPoint(1.0, [29.0]), LabeledPoint(0.0, [30.0]), LabeledPoint(7.0, [31.0]), LabeledPoint(1.0, [32.0]), LabeledPoint(6.0, [33.0]), LabeledPoint(10.0, [34.0]), LabeledPoint(12.0, [35.0]), LabeledPoint(10.0, [36.0]), LabeledPoint(14.0, [37.0]), LabeledPoint(8.0, [38.0]), LabeledPoint(8.0

In [78]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel, LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint

model = LogisticRegressionWithLBFGS.train(sc.parallelize(data), numClasses=17,iterations=5)

print model
pred = model.predict(sc.parallelize(data).map(lambda x:x.features))
#pred.take(10)
LabelsAndPredictions = sc.parallelize(data).map(lambda x:x.label).zip(pred.map(lambda x:float(x)))
LabelsAndPredictions.count()
LabelsAndPredictions.collect()


(weights=[-0.216445309046,0.0141860875959,-0.331146369124,-0.0025860096224,0.0780948790437,-0.0254469967732,-0.0677283659509,-0.00226812123473,-0.334866154162,-0.0217901781661,-0.516557913902,-0.00436871686722,-0.267289372238,-0.0230585628767,-0.400832422875,-0.014651714619,-0.442032596645,-0.0148239458731,-0.441129281282,-0.025063854846,-0.663575240151,-0.0791045413701,-0.582485321997,-0.0703117981514,-0.442692597935,-0.0723304481211,-0.403390518758,-0.0414787830528,-0.467258259796,-0.0833549444224,-0.500614027463,-0.0915874564698], intercept=0.0)


[(0.0, 3.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (2.0, 0.0),
 (1.0, 0.0),
 (1.0, 0.0),
 (5.0, 0.0),
 (9.0, 0.0),
 (8.0, 1.0),
 (8.0, 1.0),
 (9.0, 1.0),
 (13.0, 1.0),
 (1.0, 1.0),
 (2.0, 1.0),
 (2.0, 1.0),
 (1.0, 1.0),
 (7.0, 1.0),
 (2.0, 1.0),
 (1.0, 1.0),
 (4.0, 1.0),
 (7.0, 1.0),
 (1.0, 1.0),
 (0.0, 1.0),
 (7.0, 1.0),
 (1.0, 1.0),
 (6.0, 1.0),
 (10.0, 1.0),
 (12.0, 1.0),
 (10.0, 1.0),
 (14.0, 1.0),
 (8.0, 1.0),
 (8.0, 1.0),
 (0.0, 1.0),
 (6.0, 1.0),
 (1.0, 1.0),
 (6.0, 1.0),
 (1.0, 1.0),
 (0.0, 1.0),
 (0.0, 1.0),
 (1.0, 1.0),
 (0.0, 0.0),
 (0.0, 3.0),
 (0.0, 3.0),
 (0.0, 3.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (3.0, 0.0),
 (4.0, 0.0),
 (2.0, 0.0),
 (7.0, 0.0),
 (4.0, 1.0),
 (7.0, 1.0),
 (12.0, 1.0),
 (9.0, 1.0),
 (0.0, 1.0),
 (7.0, 1.0),
 (3.0, 1.0),
 (3.0, 1.0),
 (1.0, 1.0),
 (3.0, 1.0),
 (4.0, 1.0),
 (7.0, 1.0),
 (9.0, 1.0),
 (3.0, 1.0),
 (1.0, 1.0),
 (6.0,

In [82]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint

svm = SVMWithSGD.train(sc.parallelize(data), iterations=100)
pred2 = svm.predict(sc.parallelize(data).map(lambda x:x.features))
labelsAndPreds2 = sc.parallelize(data).map(lambda x:x.label).zip(pred2.map(lambda x:float(x)))
labelsAndPreds2.collect()

labelsAndPreds = sc.parallelize(data).map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(data.count())
print("Training Error = " + str(trainErr))

Py4JJavaError: An error occurred while calling o972.trainSVMModelWithSGD.
: org.apache.spark.SparkException: Input validation failed.
	at org.apache.spark.mllib.regression.GeneralizedLinearAlgorithm.run(GeneralizedLinearAlgorithm.scala:251)
	at org.apache.spark.mllib.api.python.PythonMLLibAPI.trainRegressionModel(PythonMLLibAPI.scala:94)
	at org.apache.spark.mllib.api.python.PythonMLLibAPI.trainSVMModelWithSGD(PythonMLLibAPI.scala:233)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:57)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:606)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:231)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:381)
	at py4j.Gateway.invoke(Gateway.java:259)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:133)
	at py4j.commands.Ca

In [ ]:




# Save and load model
model.save(sc, "target/tmp/pythonSVMWithSGDModel")
sameModel = SVMModel.load(sc, "target/tmp/pythonSVMWithSGDModel")

In [86]:
data = [LabeledPoint(0.0, [0.0]),LabeledPoint(1.0, [1.0]),LabeledPoint(1.0, [2.0]),LabeledPoint(1.0, [3.0])]
svm = SVMWithSGD.train(sc.parallelize(data), iterations=10)
svm.predict([1.0])
svm.predict(sc.parallelize([[1.0]])).collect()

[1]